In [6]:
from numpy.core.fromnumeric import mean
from flask import jsonify
import pandas as pd
import numpy as np
import json
import re
import http
import xgboost as xgb
import time
import urllib
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
from itertools import chain, starmap
%load_ext jupyternotify

# years completed 2002-2020
years_arr = [2002, 2003, 2004]
def download(url, num_retries=8): 
#     print('Downloading:', url)
    try: 
        
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException, http.client.IncompleteRead) as e: 
        print('Download error:', url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
        if num_retries > 0: 
            if e == http.client.IncompleteRead: 
                time.sleep(10)
                return download(url, num_retries - 1)
    return html

def flatten_json_iterative(dictionary, sep = '.', ind_start = 0):
    """Flattening a nested json file"""

    def unpack_one(parent_key, parent_value):
        """Unpack one level (only one) of nesting in json file"""
        # Unpacking one level        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                t1 = parent_key + sep + key
                yield t1, value
        elif isinstance(parent_value, list):
            i = ind_start 
            for value in parent_value:
                t2 = parent_key + sep +str(i) 
                i += 1
                yield t2, value
        else:
            yield parent_key, parent_value    

            
    # Continue iterating the unpack_one function until the terminating condition is satisfied
    while True:
        # Continue unpacking the json file until all values are atomic elements (aka neither a dictionary nor a list)
        dictionary = dict(chain.from_iterable(starmap(unpack_one, dictionary.items())))
        # Terminating condition: none of the values in the json file are a dictionary or a list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

def wbb_pbp(gameId):
        """cfb_pbp()
        Pull the game by id
        Data from API endpoints:
        * college-football/playbyplay
        * college-football/summary
        """
        # play by play
        pbp_url = "http://cdn.espn.com/core/womens-college-basketball/playbyplay?gameId={}&xhr=1&render=false&userab=18".format(gameId)
        pbp_resp = download(url=pbp_url)
        pbp_txt = {}
        
        if pbp_resp is not None:
            pbp_txt['teams'] = np.array([])
            pbp_txt['id'] = np.array([])
            pbp_txt['competitions'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['header'] = {}
            pbp_txt['pickcenter'] = np.array([])
            pbp_txt['broadcasts'] = np.array([])
            pbp_txt['videos'] = np.array([])
            pbp_txt['standings'] = np.array([])
            pbp_txt['boxscore'] = np.array([])
            pbp_txt['espnWP'] = np.array([])
            pbp_txt['gameInfo'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['timeouts'] = {}
            pbp_d = json.loads(pbp_resp)
            pbp_txt = pbp_d['gamepackageJSON']
            pbp_txt['gameId'] = pbp_d['gameId']
            # summary endpoint for pickcenter array
            summary_url = "http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/summary?event={}".format(gameId)
            summary_resp = download(url=summary_url)
            summary = json.loads(summary_resp)
            if 'pickcenter' in summary:
                summary_txt = summary['pickcenter']
            else:
                summary_txt = {}
            # ESPN's win probability
            wp = "winprobability"
            if wp in summary:
                espnWP = summary["winprobability"]
            else:
                espnWP = np.array([])

            if 'news' in pbp_txt.keys():
                del pbp_txt['news']
            if 'shop' in pbp_txt.keys():
                del pbp_txt['shop']
            pbp_txt['gameInfo'] = pbp_txt['header']['competitions'][0]
            pbp_txt['season'] = pbp_txt['header']['season']['year']
            pbp_txt['pickcenter'] = summary_txt
            pbp_txt['espnWP'] = espnWP
            # Home and Away identification variables
            homeTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['id'])
            awayTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['id'])
            homeTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['name'])
            awayTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['name'])
            homeTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['location'])
            awayTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['location'])
            homeTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['abbreviation'])
            awayTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['abbreviation'])
            homeTeamNameAlt = re.sub("Stat(.+)", "St", str(homeTeamName))
            awayTeamNameAlt = re.sub("Stat(.+)", "St", str(awayTeamName))

            if len(pbp_txt['espnWP']) > 1:
                pbp_txt['espnWP'] = espnWP
            else:
                pbp_txt['espnWP'] = espnWP
            pbp_txt['plays_mod'] = []
            for play in pbp_txt['plays']:
                p = flatten_json_iterative(play)
                pbp_txt['plays_mod'].append(p)
            pbp_txt['plays'] = pd.json_normalize(pbp_txt,'plays_mod')
            if len(pbp_txt['plays'])>1:
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                # Spread definition
                pbp_txt['plays']["homeTeamSpread"] = 2.5
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                pbp_txt['plays'] = pd.DataFrame(pbp_txt['plays'])
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']['period.number'] = pbp_txt['plays']['period.number'].apply(lambda x: int(x))
                #----- Figuring out Timeouts ---------
                pbp_txt['timeouts'] = {}
                pbp_txt['timeouts'][homeTeamId] = {"1": [], "2": []}
                pbp_txt['timeouts'][awayTeamId] = {"1": [], "2": []}
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                #----- Time ---------------
                pbp_txt['plays']['clock.mm'] = pbp_txt['plays']['clock.displayValue'].str.split(pat=':')
                pbp_txt['plays'][['clock.minutes','clock.seconds']] = pbp_txt['plays']['clock.mm'].to_list()
                pbp_txt['plays']['half'] = np.where(pbp_txt['plays']['period.number'] <= 2, "1","2")
                pbp_txt['plays']['lag_half'] = pbp_txt['plays']['half'].shift(1)
                pbp_txt['plays']['lead_half'] = pbp_txt['plays']['half'].shift(-1)

                pbp_txt['plays']['game_play_number'] = np.arange(len(pbp_txt['plays']))+1
                pbp_txt['plays']['text'] = pbp_txt['plays']['text'].astype(str)
                pbp_txt['plays']['id'] = pbp_txt['plays']['id'].apply(lambda x: int(x))
                del pbp_txt['plays']['clock.mm']
            else:
                if len(pbp_txt['pickcenter']) > 1:
                    gameSpread = pbp_txt['pickcenter'][1]['spread']
                    homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))


            pbp_json = {
                "gameId": pbp_txt['gameId'],
                "plays" : pbp_txt['plays'].to_dict(orient='records'),
                "winprobability" : np.array(pbp_txt['winprobability']).tolist(),
                "boxscore" : pbp_txt['boxscore'],
                "header" : pbp_txt['header'],
                "homeTeamSpread" : np.array(pbp_txt['homeTeamSpread']).tolist(),
                "broadcasts" : np.array(pbp_txt['broadcasts']).tolist(),
                "videos" : np.array(pbp_txt['videos']).tolist(),
                "standings" : pbp_txt['standings'],
                "pickcenter" : np.array(pbp_txt['pickcenter']).tolist(),
                "espnWP" : np.array(pbp_txt['espnWP']).tolist(),
                "gameInfo" : np.array(pbp_txt['gameInfo']).tolist(),
                "season" : np.array(pbp_txt['season']).tolist()
            }
            return pbp_json, pbp_json['season']
        else:
            
            pass
        
    
schedule = pd.read_csv('wbb_games_info_2002_2021.csv', encoding='latin-1')
schedule = schedule.sort_values(by=['season', 'date'], ascending = False)

games = schedule[(schedule['season'].isin(years_arr))&(schedule['status.type.name']=='STATUS_FINAL')].reset_index()['id']
print(f"Number of Games: {len(games)}, first gameId: {games[0]}")
g, y = wbb_pbp(gameId = games[0])
# for key in g.keys():
#     print(f"{key}: {type(g[key])}")    
json.dumps(g,indent=4)
print(g.keys())
print(g['pickcenter'])
print(g['homeTeamSpread'])
print(pd.DataFrame(g['plays']))
y

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


2021-02-26 17:54:48,377 [46120] WARNING  py.warnings:109: [JupyterRequire] C:\Users\saiem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (13,58,59,60,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Number of Games: 13341, first gameId: 240900156
dict_keys(['gameId', 'plays', 'winprobability', 'boxscore', 'header', 'homeTeamSpread', 'broadcasts', 'videos', 'standings', 'pickcenter', 'espnWP', 'gameInfo', 'season'])
{}
2.5
Empty DataFrame
Columns: []
Index: []


2004

In [ ]:
%%notify
# 2020: 2888,5077, 5091 no header
# 2019: 4960, 5220
# 2018: 2738, 3452, 4215, 4844, 4852
# 2017: 
# 2016: 3747
# 2015: 5149, 5351
# 2014: 3237, 3755
# 2013: 3306, 3602, 3873, 5258
# 2012: 3542, 3932, 5191
# 2011: 
# 2010: 3329
# 2009: 
# 2008: 290170435 290320233 290420164 283290233
# 2007: 
# 2006: 
# 2005: 
# 2004: 
# 2003: 
# 2002: 

i = 0
for game in games[i:]:
    print(f"Working on game {i+1} of {len(games)}, gameId: {games[i]}")
    if len(str(game))<9:
        i+=1
        continue
    try:
        g, y = wbb_pbp(gameId=game)
    except (TypeError) as e: 
        print("TypeError: yo", e)
        g, y = wbb_pbp(gameId=game)
    except (KeyError) as e:
        print("KeyError: yo", e)
        i+=1
        continue
    except (ValueError) as e:
        print("DecodeError: yo", e)
        i+=1
        continue
    fp = "wbb/{}/{}.json".format(y, game)
    with open(fp,'w') as f:
        json.dump(g, f, indent=4, sort_keys=False) 
#     time.sleep(1)
    i+=1

Working on game 1 of 13341, gameId: 240900156
Working on game 2 of 13341, gameId: 240880066
Working on game 3 of 13341, gameId: 240870156
Working on game 4 of 13341, gameId: 240790264
Working on game 5 of 13341, gameId: 240790204
Working on game 6 of 13341, gameId: 240790036
Working on game 7 of 13341, gameId: 240790098
Working on game 8 of 13341, gameId: 240790066
Working on game 9 of 13341, gameId: 240792603
Working on game 10 of 13341, gameId: 240782439
Working on game 11 of 13341, gameId: 240780158
Working on game 12 of 13341, gameId: 240782032
Working on game 13 of 13341, gameId: 240780235
Working on game 14 of 13341, gameId: 240782550
Working on game 15 of 13341, gameId: 240780257
Working on game 16 of 13341, gameId: 240780097
Working on game 17 of 13341, gameId: 240770006
Working on game 18 of 13341, gameId: 240772752
Working on game 19 of 13341, gameId: 240770228
Working on game 20 of 13341, gameId: 240770066
Working on game 21 of 13341, gameId: 240730149
Working on game 22 of 

Working on game 174 of 13341, gameId: 240660026
Working on game 175 of 13341, gameId: 240662239
Working on game 176 of 13341, gameId: 240660300
Working on game 177 of 13341, gameId: 240660027
Working on game 178 of 13341, gameId: 240660149
Working on game 179 of 13341, gameId: 240660099
Working on game 180 of 13341, gameId: 240662692
Working on game 181 of 13341, gameId: 240662464
Working on game 182 of 13341, gameId: 240660147
Working on game 183 of 13341, gameId: 240660120
Working on game 184 of 13341, gameId: 240660140
Working on game 185 of 13341, gameId: 240660071
Working on game 186 of 13341, gameId: 240662348
Working on game 187 of 13341, gameId: 240660024
Working on game 188 of 13341, gameId: 240662724
Working on game 189 of 13341, gameId: 240662057
Working on game 190 of 13341, gameId: 240660277
Working on game 191 of 13341, gameId: 240660219
Working on game 192 of 13341, gameId: 240662633
Working on game 193 of 13341, gameId: 240662737
Working on game 194 of 13341, gameId: 24

Working on game 335 of 13341, gameId: 240642174
Working on game 336 of 13341, gameId: 240642097
Working on game 337 of 13341, gameId: 240642011
Working on game 338 of 13341, gameId: 240640325
Working on game 339 of 13341, gameId: 240640295
Working on game 340 of 13341, gameId: 240640256
Working on game 341 of 13341, gameId: 240640048
Working on game 342 of 13341, gameId: 240642582
Working on game 343 of 13341, gameId: 240642016
Working on game 344 of 13341, gameId: 240642010
Working on game 345 of 13341, gameId: 240642116
Working on game 346 of 13341, gameId: 240642541
Working on game 347 of 13341, gameId: 240642169
Working on game 348 of 13341, gameId: 240640130
Working on game 349 of 13341, gameId: 240640231
Working on game 350 of 13341, gameId: 240642612
Working on game 351 of 13341, gameId: 240640269
Working on game 352 of 13341, gameId: 240640145
Working on game 353 of 13341, gameId: 240642608
Working on game 354 of 13341, gameId: 240640275
Working on game 355 of 13341, gameId: 24

Working on game 506 of 13341, gameId: 240592377
Working on game 507 of 13341, gameId: 240590093
Working on game 508 of 13341, gameId: 240590204
Working on game 509 of 13341, gameId: 240592132
Working on game 510 of 13341, gameId: 240590254
Working on game 511 of 13341, gameId: 240590108
Working on game 512 of 13341, gameId: 240590055
Working on game 513 of 13341, gameId: 240592154
Working on game 514 of 13341, gameId: 240592754
Working on game 515 of 13341, gameId: 240592630
Working on game 516 of 13341, gameId: 240592534
Working on game 517 of 13341, gameId: 240592523
Working on game 518 of 13341, gameId: 240592351
Working on game 519 of 13341, gameId: 240590349
Working on game 520 of 13341, gameId: 240590265
Working on game 521 of 13341, gameId: 240590085
Working on game 522 of 13341, gameId: 240590068
Working on game 523 of 13341, gameId: 240592739
Working on game 524 of 13341, gameId: 240592717
Working on game 525 of 13341, gameId: 240592681
Working on game 526 of 13341, gameId: 24

Working on game 667 of 13341, gameId: 240560201
Working on game 668 of 13341, gameId: 240562306
Working on game 669 of 13341, gameId: 240560087
Working on game 670 of 13341, gameId: 240560046
Working on game 671 of 13341, gameId: 240560041
Working on game 672 of 13341, gameId: 240562649
Working on game 673 of 13341, gameId: 240562619
Working on game 674 of 13341, gameId: 240562599
Working on game 675 of 13341, gameId: 240562550
Working on game 676 of 13341, gameId: 240562378
Working on game 677 of 13341, gameId: 240562309
Working on game 678 of 13341, gameId: 240562026
Working on game 679 of 13341, gameId: 240560399
Working on game 680 of 13341, gameId: 240560311
Working on game 681 of 13341, gameId: 240560276
Working on game 682 of 13341, gameId: 240560195
Working on game 683 of 13341, gameId: 240560193
Working on game 684 of 13341, gameId: 240560042
Working on game 685 of 13341, gameId: 240562352
Working on game 686 of 13341, gameId: 240560326
Working on game 687 of 13341, gameId: 24

...

Working on game 1628 of 13341, gameId: 240330290
Working on game 1629 of 13341, gameId: 240332400
Working on game 1630 of 13341, gameId: 240332029
Working on game 1631 of 13341, gameId: 240332010
Working on game 1632 of 13341, gameId: 240332569
Working on game 1633 of 13341, gameId: 240332448
Working on game 1634 of 13341, gameId: 240332415
Working on game 1635 of 13341, gameId: 240332379
Working on game 1636 of 13341, gameId: 240332154
Working on game 1637 of 13341, gameId: 240330232
Working on game 1638 of 13341, gameId: 240320275
Working on game 1639 of 13341, gameId: 240320251
Working on game 1640 of 13341, gameId: 240320154
Working on game 1641 of 13341, gameId: 240322633
Working on game 1642 of 13341, gameId: 240320099
Working on game 1643 of 13341, gameId: 240320238
Working on game 1644 of 13341, gameId: 240320356
Working on game 1645 of 13341, gameId: 240320344
Working on game 1646 of 13341, gameId: 240320145
Working on game 1647 of 13341, gameId: 240320044
Working on game 1648

Working on game 1786 of 13341, gameId: 240312341
Working on game 1787 of 13341, gameId: 240310160
Working on game 1788 of 13341, gameId: 240310087
Working on game 1789 of 13341, gameId: 240302751
Working on game 1790 of 13341, gameId: 240302612
Working on game 1791 of 13341, gameId: 240300150
Working on game 1792 of 13341, gameId: 240302561
Working on game 1793 of 13341, gameId: 240302230
Working on game 1794 of 13341, gameId: 240302168
Working on game 1795 of 13341, gameId: 240300219
Working on game 1796 of 13341, gameId: 240300172
Working on game 1797 of 13341, gameId: 240300171
Working on game 1798 of 13341, gameId: 240300163
Working on game 1799 of 13341, gameId: 240302363
Working on game 1800 of 13341, gameId: 240302329
Working on game 1801 of 13341, gameId: 240302083
Working on game 1802 of 13341, gameId: 240300044
Working on game 1803 of 13341, gameId: 240302426
Working on game 1804 of 13341, gameId: 240290062
Working on game 1805 of 13341, gameId: 240290299
Working on game 1806

Working on game 1954 of 13341, gameId: 240250008
Working on game 1955 of 13341, gameId: 240250248
Working on game 1956 of 13341, gameId: 240250333
Working on game 1957 of 13341, gameId: 240250269
Working on game 1958 of 13341, gameId: 240250254
Working on game 1959 of 13341, gameId: 240250238
Working on game 1960 of 13341, gameId: 240250139
Working on game 1961 of 13341, gameId: 240250194
Working on game 1962 of 13341, gameId: 240250084
Working on game 1963 of 13341, gameId: 240250066
Working on game 1964 of 13341, gameId: 240252670
Working on game 1965 of 13341, gameId: 240252603
Working on game 1966 of 13341, gameId: 240252579
Working on game 1967 of 13341, gameId: 240252368
Working on game 1968 of 13341, gameId: 240252363
Working on game 1969 of 13341, gameId: 240250295
Working on game 1970 of 13341, gameId: 240250228
Working on game 1971 of 13341, gameId: 240250227
Working on game 1972 of 13341, gameId: 240250151
Working on game 1973 of 13341, gameId: 240252182
Working on game 1974

Working on game 2122 of 13341, gameId: 240222464
Working on game 2123 of 13341, gameId: 240220166
Working on game 2124 of 13341, gameId: 240222172
Working on game 2125 of 13341, gameId: 240220328
Working on game 2126 of 13341, gameId: 240220254
Working on game 2127 of 13341, gameId: 240220252
Working on game 2128 of 13341, gameId: 240220068
Working on game 2129 of 13341, gameId: 240222623
Working on game 2130 of 13341, gameId: 240222181
Working on game 2131 of 13341, gameId: 240220156
Working on game 2132 of 13341, gameId: 240220079
Working on game 2133 of 13341, gameId: 240220008
Working on game 2134 of 13341, gameId: 240220145
Working on game 2135 of 13341, gameId: 240220077
Working on game 2136 of 13341, gameId: 240220002
Working on game 2137 of 13341, gameId: 240222393
Working on game 2138 of 13341, gameId: 240222320
Working on game 2139 of 13341, gameId: 240220356
Working on game 2140 of 13341, gameId: 240220344
Working on game 2141 of 13341, gameId: 240220275
Working on game 2142

Working on game 2290 of 13341, gameId: 240182363
Working on game 2291 of 13341, gameId: 240182325
Working on game 2292 of 13341, gameId: 240180119
Working on game 2293 of 13341, gameId: 240180104
Working on game 2294 of 13341, gameId: 240180059
Working on game 2295 of 13341, gameId: 240180048
Working on game 2296 of 13341, gameId: 240170062
Working on game 2297 of 13341, gameId: 240172439
Working on game 2298 of 13341, gameId: 240170027
Working on game 2299 of 13341, gameId: 240172608
Working on game 2300 of 13341, gameId: 240170331
Working on game 2301 of 13341, gameId: 240172540
Working on game 2302 of 13341, gameId: 240172539
Working on game 2303 of 13341, gameId: 240172502
Working on game 2304 of 13341, gameId: 240172492
Working on game 2305 of 13341, gameId: 240172463
Working on game 2306 of 13341, gameId: 240172239
Working on game 2307 of 13341, gameId: 240170279
Working on game 2308 of 13341, gameId: 240170023
Working on game 2309 of 13341, gameId: 240172692
Working on game 2310